#  Logistic Regression Code Along Project
## Titanic Dataset

We'll attempt to predict what passengers survived the Titanic crash based solely on passenger's features (age, cabin children, etc..)

In [1]:
import findspark
findspark.init('/home/fernando/spark-2.4.6-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('titanic').getOrCreate()

In [2]:
df = spark.read.csv('Logistic_Regression/titanic.csv',inferSchema=True,header=True)

In [6]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## Get only useful features:

In [4]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [5]:
my_cols = df.select([
    'Survived',
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'Embarked'
])

## Get with missing data
Easiest way: Drop it

In [6]:
my_final_data = my_cols.na.drop()

##  Working with categorical columns

In [7]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [8]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex') # A B C---> 0, 1, 2
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec') # A ---> [1, 0, 0]

In [10]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [11]:
assembler = VectorAssembler(inputCols=[
    'Pclass',
    'SexVec',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'EmbarkVec'],
    outputCol='features')

In [12]:
from pyspark.ml.classification import LogisticRegression

# Pipelines
Set stages for different tasks

In [13]:
from pyspark.ml import Pipeline

In [14]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [15]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

## Split data and train the pipeline (model)

In [16]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [17]:
fit_model = pipeline.fit(train_titanic_data)

In [18]:
results = fit_model.transform(test_titanic_data)

In [24]:
results.select(['Survived', 'prediction']).show(10)

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 10 rows



## Evaluate

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

### Area Under the Curve

In [25]:
AUC = my_eval.evaluate(results)

In [26]:
AUC

0.7807692307692308